# EY Data Science Challange

Goal: Predict whether the last trajectory enter the area or not <br>
In this notebook, we only train the model with feature related to the **last instance**.
This will serve as our baseline.

In [1]:
import pandas as pd
train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')

# Data Preporcessing(Mapped bag space method)
In bag-based multi instance learning, we will first create a mapping for the bag. In this notebook, we will explore different type of mapping method. We first pick out the last row of the data because the missing value of the last row in test set will create a disperse on the assumption of model building.

In [2]:
# the target area
x_target = [3750901.5068,3770901.5068]
y_target  = [-19268905.6133,-19208905.6133]

from utility import create_response
train_last = train.groupby('hash').last() #pick the last trajectory out
train_last['target'] = create_response(train_last.x_exit,train_last.y_exit,x_target,y_target)
train_last.head()

,Unnamed: 0,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,target
hash,,,,,,,,,,,,
0000a8602cf2def930488dee7cdad104_1,5,traj_0000a8602cf2def930488dee7cdad104_1_5,15:02:31,15:18:33,NaN,NaN,NaN,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,0
0000cf177130469eeac79f67b6bcf3df_9,9,traj_0000cf177130469eeac79f67b6bcf3df_9_3,15:00:32,15:29:48,1.149404,1.149404,1.149404,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,0
0001f97b99a80f18f62e2d44e54ef33d_3,11,traj_0001f97b99a80f18f62e2d44e54ef33d_3_1,14:34:35,15:19:51,30.167742,30.167742,30.167742,3.758738e+06,-1.937594e+07,3.769687e+06,-1.914258e+07,0
0002124248b0ca510dea42824723ccac_31,20,traj_0002124248b0ca510dea42824723ccac_31_10,15:28:54,15:28:54,NaN,NaN,NaN,3.767866e+06,-1.917797e+07,3.767866e+06,-1.917797e+07,0
000219c2a6380c307e8bffd85b5e404b_23,28,traj_000219c2a6380c307e8bffd85b5e404b_23_16,15:08:05,15:08:05,NaN,NaN,NaN,3.747641e+06,-1.922695e+07,3.747641e+06,-1.922695e+07,0


## Mapping method based on bag statistics

#### Create Variables Delta Time
We cacluated the differences between time_entry and time_exit to figure out how the long did the user travel

In [3]:
#calculate delta_time for the last trajectory
from utility import delta_t
train['delta_time'] = delta_t(train.time_entry,train.time_exit)
time_target = '15:00:00'
train['time_from_target'] = delta_t(train.time_entry,time_target)
train.head()

,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,delta_time,time_from_target
0,0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,07:04:31,07:08:32,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,-241.0,-28529.0
1,1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,07:20:34,07:25:42,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,-308.0,-27566.0
2,2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,07:53:32,08:03:25,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,-593.0,-25588.0
3,3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,08:17:50,08:37:23,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,-1173.0,-24130.0
4,4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,14:38:09,14:38:09,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,0.0,-1311.0


In [4]:
# calculate the delta t and time from target for the last instance
train_last['delta_time'] = delta_t(train_last.time_entry,train_last.time_exit)
train_last['time_from_target'] = delta_t(train_last.time_entry,time_target)
train_last.head()

,Unnamed: 0,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,target,delta_time,time_from_target
hash,,,,,,,,,,,,,,
0000a8602cf2def930488dee7cdad104_1,5,traj_0000a8602cf2def930488dee7cdad104_1_5,15:02:31,15:18:33,NaN,NaN,NaN,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,0,-962.0,151.0
0000cf177130469eeac79f67b6bcf3df_9,9,traj_0000cf177130469eeac79f67b6bcf3df_9_3,15:00:32,15:29:48,1.149404,1.149404,1.149404,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,0,-1756.0,32.0
0001f97b99a80f18f62e2d44e54ef33d_3,11,traj_0001f97b99a80f18f62e2d44e54ef33d_3_1,14:34:35,15:19:51,30.167742,30.167742,30.167742,3.758738e+06,-1.937594e+07,3.769687e+06,-1.914258e+07,0,-2716.0,-1525.0
0002124248b0ca510dea42824723ccac_31,20,traj_0002124248b0ca510dea42824723ccac_31_10,15:28:54,15:28:54,NaN,NaN,NaN,3.767866e+06,-1.917797e+07,3.767866e+06,-1.917797e+07,0,0.0,1734.0
000219c2a6380c307e8bffd85b5e404b_23,28,traj_000219c2a6380c307e8bffd85b5e404b_23_16,15:08:05,15:08:05,NaN,NaN,NaN,3.747641e+06,-1.922695e+07,3.747641e+06,-1.922695e+07,0,0.0,485.0


#### Create Variable: Euclidean Distance from Center
Calculating the distance between the entry point and the center of the target area.

In [5]:
# the target area
x_min,x_max = [3750901.5068,3770901.5068]
y_min,y_max  = [-19268905.6133,-19208905.6133]

# the center of target area
x_target = (x_max + x_min)/2
y_target = (y_max + y_min)/2

# calculate the euclidean 
from utility import eu_dist
train['dist_from_center'] = eu_dist(train.x_entry,train.y_entry,x_target,y_target)                                           
# train['delta_dist'] =  eu_dist(train.x_entry,train.y_entry,train.x_exit,train.y_exit)
train.head()

,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,delta_time,time_from_target,dist_from_center
0,0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,07:04:31,07:08:32,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,-241.0,-28529.0,145261.881023
1,1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,07:20:34,07:25:42,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,-308.0,-27566.0,85266.551966
2,2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,07:53:32,08:03:25,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,-593.0,-25588.0,56962.437581
3,3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,08:17:50,08:37:23,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,-1173.0,-24130.0,55736.470981
4,4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,14:38:09,14:38:09,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,0.0,-1311.0,49342.931074


In [6]:
#calculate the distance from center for the last trajectory
train_last['dist_from_center'] = eu_dist(train_last.x_entry,train_last.y_entry,x_target,y_target)    

### Average Mapping

In [7]:
train = train.groupby('hash').mean()
new_columns = ['x_entry_last','y_entry_last','delta_time_last','time_from_target_last','distance_from_center_last']
train[new_columns] = train_last[['x_entry','y_entry','delta_time','time_from_target','dist_from_center']]
train['target']=train_last["target"]
train.head()

,Unnamed: 0,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,delta_time,time_from_target,dist_from_center,x_entry_last,y_entry_last,delta_time_last,time_from_target_last,distance_from_center_last,target
hash,,,,,,,,,,,,,,,,,
0000a8602cf2def930488dee7cdad104_1,2.5,NaN,NaN,NaN,3.745759e+06,-1.926162e+07,3.745770e+06,-1.926773e+07,-546.166667,-17828.833333,73061.375809,3.744945e+06,-1.928183e+07,-962.0,151.0,45797.982227,0
0000cf177130469eeac79f67b6bcf3df_9,7.5,0.414303,0.414303,0.414303,3.749168e+06,-1.926617e+07,3.749298e+06,-1.926614e+07,-439.000000,-900.500000,29686.029755,3.749088e+06,-1.926605e+07,-1756.0,32.0,29603.985176,0
0001f97b99a80f18f62e2d44e54ef33d_3,10.5,31.181998,31.181998,31.181998,3.765100e+06,-1.924004e+07,3.763345e+06,-1.921978e+07,-1809.000000,-6849.500000,136117.999334,3.758738e+06,-1.937594e+07,-2716.0,-1525.0,137051.659155,0
0002124248b0ca510dea42824723ccac_31,16.0,NaN,NaN,NaN,3.763971e+06,-1.919657e+07,3.764898e+06,-1.919264e+07,-142.444444,-6029.111111,42606.722013,3.767866e+06,-1.917797e+07,0.0,1734.0,61336.955341,0
000219c2a6380c307e8bffd85b5e404b_23,24.5,NaN,NaN,NaN,3.752336e+06,-1.923014e+07,3.752809e+06,-1.923513e+07,-456.625000,-34069.125000,12923.732799,3.747641e+06,-1.922695e+07,0.0,485.0,17851.785279,0


In [8]:
# normalize the variables
columns = ['delta_time','time_from_target','dist_from_center','x_entry',
           'y_entry','x_entry_last','y_entry_last','delta_time_last','time_from_target_last','distance_from_center_last']
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
train[columns] =  min_max_scaler.fit_transform(train[columns].values)
train.head()

,Unnamed: 0,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,delta_time,time_from_target,dist_from_center,x_entry_last,y_entry_last,delta_time_last,time_from_target_last,distance_from_center_last,target
hash,,,,,,,,,,,,,,,,,
0000a8602cf2def930488dee7cdad104_1,2.5,NaN,NaN,NaN,0.130960,0.348871,3.745770e+06,-1.926773e+07,0.944075,0.585602,0.379578,0.108488,0.287755,0.949739,0.823986,0.238078,0
0000cf177130469eeac79f67b6bcf3df_9,7.5,0.414303,0.414303,0.414303,0.225903,0.335049,3.749298e+06,-1.926614e+07,0.955048,0.914256,0.152975,0.223364,0.335615,0.908255,0.817913,0.153702,0
0001f97b99a80f18f62e2d44e54ef33d_3,10.5,31.181998,31.181998,31.181998,0.669647,0.414392,3.763345e+06,-1.921978e+07,0.814766,0.798760,0.709000,0.490926,0.002400,0.858098,0.738454,0.713536,0
0002124248b0ca510dea42824723ccac_31,16.0,NaN,NaN,NaN,0.638209,0.546348,3.764898e+06,-1.919264e+07,0.985414,0.814687,0.220476,0.744012,0.602711,1.000000,0.904772,0.319040,0
000219c2a6380c307e8bffd85b5e404b_23,24.5,NaN,NaN,NaN,0.314147,0.444440,3.752809e+06,-1.923513e+07,0.953243,0.270306,0.065405,0.183241,0.454165,1.000000,0.841031,0.092470,0


In [57]:
from sklearn.utils import resample
# Separate majority and minority classes
train_majority = train[train.target==0]
train_minority = train[train.target==1]

print(train_minority.shape[0])

# downsample majority class
train_majority_downsampled = resample(train_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples= int(36380*2),     # to match minority class
                                 random_state=123)
 
# Combine majority class with upsampled minority class
train_downsampled = pd.concat([train_minority, train_majority_downsampled])
 
# Display new class counts
train_downsampled.target.value_counts()

36380


0    72760
1    36380
Name: target, dtype: int64

In [9]:
print(train.columns) #read the column names for training data set
X = train[columns]
y = train['target']
# X = train_downsampled[columns]
# y = train_downsampled['target']

Index(['Unnamed: 0', 'vmax', 'vmin', 'vmean', 'x_entry', 'y_entry', 'x_exit',
       'y_exit', 'delta_time', 'time_from_target', 'dist_from_center',
       'x_entry_last', 'y_entry_last', 'delta_time_last',
       'time_from_target_last', 'distance_from_center_last', 'target'],
      dtype='object')


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [11]:
y_test.mean()

0.2695707306157461

## Model Training Process

In [12]:
# model training
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
clf.fit(X_train, y_train)
print(clf.score(X_test,y_test))

0.878193413642636


In [13]:
from sklearn.neighbors import KNeighborsClassifier
# for k in range(85,95,1):
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)
print(neigh.score(X_test,y_test))

0.9205236265990377


In [14]:
from sklearn.neural_network import MLPClassifier

MLP = MLPClassifier(hidden_layer_sizes=(10),random_state=121)
MLP.fit(X_train, y_train)
print(MLP.score(X_test,y_test))

0.9164584343415507


/Users/garylau/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [68]:
from sklearn import tree
dtree = tree.DecisionTreeClassifier(max_depth=4)
dtree.fit(X_train, y_train)
print(dtree.score(X_test,y_test))

0.9220267546270845


In [69]:
from sklearn.svm import SVC
svm = SVC(random_state=0, tol=1e-5, gamma= 'scale')
svm.fit(X_train, y_train)

print(svm.score(X_test,y_test))

0.9051218618288437


# Ensembling Technique

In [70]:
# from sklearn import tree
# from sklearn.ensemble import BaggingClassifier
# from sklearn.model_selection import cross_validate
# dtree = tree.DecisionTreeClassifier()
# bagging = BaggingClassifier(dtree,random_state=0)
# bagging.fit(X_train, y_train)
# print(bagging.score(X_test,y_test))

In [71]:
# from sklearn.ensemble import RandomForestClassifier
# rtree = RandomForestClassifier(n_estimators=10,max_depth=5)
# rtree.fit(X_train,y_train)
# print(rtree.score(X_test,y_test))

In [72]:
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('logistic', clf), ('knn', neigh), ('mlp', MLP),('dtree',dtree),('svm',svm)], voting='hard')
eclf1.fit(X_train, y_train)
eclf1.score(X_test,y_test)

/Users/garylau/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.9145134689389774

In [15]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(base_estimator=clf, n_estimators=50, learning_rate=1.0,
                         algorithm='SAMME', random_state=0)
ada.fit(X_train,y_train)
ada.score(X_test,y_test)

0.8308656248834521

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

gbc= GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=3, random_state=0).fit(X_train, y_train)
gbc.score(X_test,y_test)

0.9348077425129602

# Produce Result

In [20]:
# test_bag = test_da.set_index('trajectory_id')
# test_bag = test_bag.drop(test_data[['hash','trajectory_id']].groupby('hash').last().values.flatten(),axis=0) # drop the last column
# test_bag.head()

In [27]:
test.head()

,Unnamed: 0,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,delta_time,dist_from_center,time_from_target,x_entry_last,y_entry_last,delta_time_last,time_from_target_last,distance_from_center_last,trajectory_id,target
hash,,,,,,,,,,,,,,,,,,
00032f51796fd5437b238e3a9823d13d_31,2.0,0.00,0.00,0.00,0.844317,0.710883,3.772799e+06,-1.913605e+07,0.888601,0.504684,0.798821,0.892689,0.703353,0.970270,0.780613,0.493849,traj_00032f51796fd5437b238e3a9823d13d_31_5,0
000479418b5561ab694a2870cc04fd43_25,6.0,17.29,17.29,17.29,0.831442,0.124639,3.769958e+06,-1.934180e+07,0.951763,0.507076,0.718031,0.805297,0.107299,0.992426,0.880140,0.535906,traj_000479418b5561ab694a2870cc04fd43_25_10,0
000506a39775e5bca661ac80e3f466eb_29,9.0,NaN,NaN,NaN,0.469144,0.637716,3.757330e+06,-1.912208e+07,0.985785,0.381353,0.707618,0.457156,0.419087,1.000000,0.868419,0.017412,traj_000506a39775e5bca661ac80e3f466eb_29_5,1
0005401ceddaf27a9b7f0d42ef1fbe95_1,12.0,NaN,NaN,NaN,0.371564,0.455540,3.751338e+06,-1.916260e+07,0.982361,0.469046,0.684392,0.541667,0.065921,0.971190,0.904034,0.604949,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0
00063a4f6c12e1e4de7d876580620667_3,15.5,NaN,NaN,NaN,0.710093,0.525907,3.772288e+06,-1.915435e+07,0.848269,0.295070,0.776905,0.703470,0.626812,0.952785,0.744026,0.359256,traj_00063a4f6c12e1e4de7d876580620667_3_4,0


In [28]:
# test = test.groupby('hash').last()
# test = test.fillna(method='bfill')
test['delta_time'] = delta_t(test.time_entry,test.time_exit)
test['dist_from_center'] = eu_dist(test.x_entry,test.y_entry,x_target,y_target)
test['time_from_target'] = delta_t(test.time_entry,time_target)
# test['delta_dist'] = eu_dist(test.x_entry,test.y_entry,x_target,y_target)
test_last = test.groupby('hash').last()
test = test.groupby('hash').mean()
print(test.shape)
print(test_last.shape)
# test[new_columns] = test_last[['x_entry','y_entry','delta_time','time_from_target','dist_from_center']]
# test['trajectory_id'] = test_last['trajectory_id']

AttributeError: 'DataFrame' object has no attribute 'time_entry'

In [29]:
test[new_columns] = test_last[['x_entry','y_entry','delta_time','time_from_target','dist_from_center']]
test['trajectory_id'] = test_last['trajectory_id']
test[columns] =  min_max_scaler.fit_transform(test[columns].values)
test.head()

,Unnamed: 0,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,delta_time,dist_from_center,time_from_target,x_entry_last,y_entry_last,delta_time_last,time_from_target_last,distance_from_center_last,trajectory_id,target
hash,,,,,,,,,,,,,,,,,,
00032f51796fd5437b238e3a9823d13d_31,2.0,0.00,0.00,0.00,0.844317,0.710883,3.772799e+06,-1.913605e+07,0.888601,0.504684,0.798821,0.892689,0.703353,0.970270,0.780613,0.493849,traj_00032f51796fd5437b238e3a9823d13d_31_5,0
000479418b5561ab694a2870cc04fd43_25,6.0,17.29,17.29,17.29,0.831442,0.124639,3.769958e+06,-1.934180e+07,0.951763,0.507076,0.718031,0.805297,0.107299,0.992426,0.880140,0.535906,traj_000479418b5561ab694a2870cc04fd43_25_10,0
000506a39775e5bca661ac80e3f466eb_29,9.0,NaN,NaN,NaN,0.469144,0.637716,3.757330e+06,-1.912208e+07,0.985785,0.381353,0.707618,0.457156,0.419087,1.000000,0.868419,0.017412,traj_000506a39775e5bca661ac80e3f466eb_29_5,1
0005401ceddaf27a9b7f0d42ef1fbe95_1,12.0,NaN,NaN,NaN,0.371564,0.455540,3.751338e+06,-1.916260e+07,0.982361,0.469046,0.684392,0.541667,0.065921,0.971190,0.904034,0.604949,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0
00063a4f6c12e1e4de7d876580620667_3,15.5,NaN,NaN,NaN,0.710093,0.525907,3.772288e+06,-1.915435e+07,0.848269,0.295070,0.776905,0.703470,0.626812,0.952785,0.744026,0.359256,traj_00063a4f6c12e1e4de7d876580620667_3_4,0


In [21]:
X_result = test[columns]
# X_result = X_result.fillna(method='bfill')
# test['target'] = clf.predict(X_result)
# test['target'] = neigh.predict(X_result)
# test['target'] = dtree.predict(X_result)
# test['target'] = rtree.predict(X_result)
# test['target'] = bagging.predict(X_result)
# test['target'] = svm.predict(X_result)
# test['target'] = eclf1.predict(X_result)
test['target'] = gbc.predict(X_result)
test.target.mean()

0.2786513501417276

In [30]:
test.head()

,Unnamed: 0,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,delta_time,dist_from_center,time_from_target,x_entry_last,y_entry_last,delta_time_last,time_from_target_last,distance_from_center_last,trajectory_id,target
hash,,,,,,,,,,,,,,,,,,
00032f51796fd5437b238e3a9823d13d_31,2.0,0.00,0.00,0.00,0.844317,0.710883,3.772799e+06,-1.913605e+07,0.888601,0.504684,0.798821,0.892689,0.703353,0.970270,0.780613,0.493849,traj_00032f51796fd5437b238e3a9823d13d_31_5,0
000479418b5561ab694a2870cc04fd43_25,6.0,17.29,17.29,17.29,0.831442,0.124639,3.769958e+06,-1.934180e+07,0.951763,0.507076,0.718031,0.805297,0.107299,0.992426,0.880140,0.535906,traj_000479418b5561ab694a2870cc04fd43_25_10,0
000506a39775e5bca661ac80e3f466eb_29,9.0,NaN,NaN,NaN,0.469144,0.637716,3.757330e+06,-1.912208e+07,0.985785,0.381353,0.707618,0.457156,0.419087,1.000000,0.868419,0.017412,traj_000506a39775e5bca661ac80e3f466eb_29_5,1
0005401ceddaf27a9b7f0d42ef1fbe95_1,12.0,NaN,NaN,NaN,0.371564,0.455540,3.751338e+06,-1.916260e+07,0.982361,0.469046,0.684392,0.541667,0.065921,0.971190,0.904034,0.604949,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0
00063a4f6c12e1e4de7d876580620667_3,15.5,NaN,NaN,NaN,0.710093,0.525907,3.772288e+06,-1.915435e+07,0.848269,0.295070,0.776905,0.703470,0.626812,0.952785,0.744026,0.359256,traj_00063a4f6c12e1e4de7d876580620667_3_4,0


In [31]:
result = test.reset_index()[['trajectory_id','target']]
result.columns = ['id','target']
# result = result.set_index('id')
result.head()

,id,target
0,traj_00032f51796fd5437b238e3a9823d13d_31_5,0
1,traj_000479418b5561ab694a2870cc04fd43_25_10,0
2,traj_000506a39775e5bca661ac80e3f466eb_29_5,1
3,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0
4,traj_00063a4f6c12e1e4de7d876580620667_3_4,0


In [32]:
test_last.shape

(33515, 14)

In [33]:
#write result as csv
result.to_csv('result.csv',sep=',')